In [2]:
pip install azure-cognitiveservices-vision-customvision

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install --force-reinstall charset-normalizer==3.1.0

  Using cached charset_normalizer-3.1.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (195 kB)
  Attempting uninstall: charset-normalizer
    Found existing installation: charset-normalizer 3.1.0
    Uninstalling charset-normalizer-3.1.0:
      Successfully uninstalled charset-normalizer-3.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.3.2 requires dill<0.3.6, but you have dill 0.3.6 which is incompatible.
tensorboard 2.2.2 requires google-auth<2,>=1.6.3, but you have google-auth 2.16.0 which is incompatible.
tensorflow 2.2.1 requires h5py<2.11.0,>=2.10.0, but you have h5py 3.8.0 which is incompatible.
tensorflow 2.2.1 requires numpy<1.19.0,>=1.16.0, but you have numpy 1.21.6 which is incompatible.
tensorflow-cpu 2.2.1 requires h5py<2.11.0,>=2.10.0, but you have h5py 3.8.0 which is incompatible.
tensorflow-cpu 2.2.1 requires numpy<1.1

In [ ]:
%pip install azure-cognitiveservices-vision-customvision
%pip install pylabel

In [1]:
#Import Azure cognitive services libraries 
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials

#Import other libraries used in this notebook 
import os, zipfile
from pathlib import PurePath
from os.path import exists
from decimal import *

from pylabel import importer

In [2]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials
import os, time, uuid

In [3]:
# Replace with valid values
ENDPOINT = "https://costomvision00.cognitiveservices.azure.com/"
training_key = "02706883018a49278a16d2821645f9e0"
prediction_key = "f0d1833881a84470be898363d6b705cb"
prediction_resource_id = "/subscriptions/78be0479-f85e-40c8-80e7-dbbac5089527/resourceGroups/T-20-Q4/providers/Microsoft.CognitiveServices/accounts/costomvision00-Prediction"

In [4]:
credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(ENDPOINT, prediction_credentials)

In [5]:
publish_iteration_name = "tagexp"

# Find the object detection domain
obj_detection_domain = next(domain for domain in trainer.get_domains() if domain.type == "ObjectDetection" and domain.name == "General")

# Use uuid to avoid project name collisions.
project = trainer.create_project("Animal Dataset", domain_id=obj_detection_domain.id)

## load dataset to ML studio

In [56]:
#only run once
# Import data into AML Studio

from azure.storage.blob import BlobServiceClient

#%%capture
os.makedirs("data/trainingimages/", exist_ok=True)

# Set up Blob Storage client and download the ZIP file
storage_account_name = "t20storageq4"
container_name = "trainingimages"
blob_name = "test.zip"
sas_token = "?sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupiytfx&se=2023-06-30T10:58:10Z&st=2023-05-11T02:58:10Z&spr=https,http&sig=u5JzrO3V2ohj0PoSMjfQrTWnq2LUH3XJzl%2Fz7O5ftx8%3D"

blob_service_client = BlobServiceClient(account_url=f"https://{storage_account_name}.blob.core.windows.net", credential=sas_token)
container_client = blob_service_client.get_container_client(container_name)
blob_client = container_client.get_blob_client(blob_name)

KeyboardInterrupt: 

In [58]:
#only run once
os.makedirs("data/extract_trainingimages/", exist_ok=True)

# Download the ZIP file to the notebook's local filesystem
local_file_path = "data/trainingimages/test.zip"
with open(local_file_path, "wb") as file:
    file.write(blob_client.download_blob().readall())

extract_dir = "data/extract_trainingimages/"
with zipfile.ZipFile(local_file_path, "r") as zip_ref:
    zip_ref.extractall(extract_dir)

## rearrange data structure

In [6]:
#only run once

import os, shutil
from tqdm import tqdm

dataset_path = "data/extract_trainingimages/"


for animal_category in tqdm(os.listdir(dataset_path)):
    category_dir = os.path.join(dataset_path, animal_category)
    label_dir = os.path.join(category_dir, "Label")

    for text_file in os.listdir(label_dir):
        if text_file.endswith('.txt'):
            text_src = os.path.join(label_dir, text_file)
            text_dst = "data/new_train/labels/"

            file_path = os.path.join(text_dst, text_file)
            if os.path.isfile(file_path) is False:
                shutil.move(text_src, text_dst)
    
    for image_file in os.listdir(category_dir):
        if image_file.endswith(".jpg"):
            image_src = os.path.join(category_dir, image_file)
            image_dst = "data/new_train/images/"

            image_path = os.path.join(image_dst, image_file)
            if os.path.isfile(image_path) is False:
                shutil.move(image_src, image_dst)

100%|██████████| 80/80 [01:02<00:00,  1.28it/s]


In [6]:
dataset_path = "data/extract_trainingimages/"
categories = os.listdir(dataset_path)

cate_dic = {}
for i, name in enumerate(categories):
  cate_dic[name] = i
 


In [ ]:
#only run once

import os

directory = "data/new_train/labels"

for txt_file in os.listdir(directory):
    if txt_file.endswith('.txt'):
        file_path = os.path.join(directory, txt_file)

        with open(file_path, "r") as f:
            n = 0
            for line in f:
                n += 1
                line_list = line.split()
                ymax = line_list.pop()
                xmax = line_list.pop()
                ymin = line_list.pop()
                xmin = line_list.pop()

                cate_str = ""
                for s in line_list:
                    cate_str += s
                    cate_str += " "
                cate_str = cate_str.rstrip()
                print(cate_str)
                #print(cate_dic[cate_str])
                cate_id = cate_dic[cate_str]

                if n == 1: 
                    with open(file_path, "w") as f1:
                        f1.write(str(cate_id) + " " + xmin + " " + ymin + " " + xmax + " " + ymax + "\n")
                else:
                    with open(file_path, "a") as f1:
                        f1.write(str(cate_id) + " " + xmin + " " + ymin + " " + xmax + " " + ymax + "\n")
                
        f.close()

        with open(file_path, "r") as f2:
            print(txt_file)

            for line2 in f2:
                print(line2.split())

In [7]:
import cv2 as cv
import cv2

In [ ]:
#run once
os.mkdir("yolo")
os.mkdir("yolo/train")
os.mkdir("yolo/train/images")
os.mkdir("yolo/train/labels")

In [ ]:
# run once
size = (640,640)
directory = "data/new_train/images"
for img_file in os.listdir(directory):
    if img_file.endswith('.jpg'):
        file_path = os.path.join(directory, img_file)
        img = cv2.imread(file_path, cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, size)
        cv2.imwrite("yolo/train/images/"+img_file, img) 




In [11]:
#run once

directory = "data/new_train/labels"
yolo_p = "yolo/train/labels"

for txt_file in os.listdir(directory):
    if txt_file.endswith('.txt'):
        file_path = os.path.join(directory, txt_file)
        destination_path = os.path.join(yolo_p,txt_file)
        with open(file_path, "r") as source:
            with open(destination_path, "w") as destination:
                image_file_name_no_ext = txt_file[0:len(txt_file)-4]
                img = cv2.imread("data/new_train/images/"+image_file_name_no_ext+".jpg", cv2.IMREAD_COLOR)
                height = img.shape[0]
                width = img.shape[1]
                for line in source:
                    labeling_data = line.split()
                    
                    xmin = float(labeling_data[1])
                    ymin = float(labeling_data[2])
                    xmax = float(labeling_data[3])
                    ymax = float(labeling_data[4])

                    cx = (xmin + xmax)/2.0/width
                    cy = (ymin + ymax)/2.0/height
                    box_width = (xmax - xmin)/width
                    box_height = (ymax - ymin)/height
                    
                    destination.write(str(labeling_data[0])+" ")
                    destination.write(str(cx)+" ")
                    destination.write(str(cy)+" ")
                    destination.write(str(box_width)+" ")
                    destination.write(str(box_height)+"\n")


## load data to dataset object


In [13]:
#Import annotations as a PyLabel dataset
categories = os.listdir(dataset_path)

dataset = importer.ImportYoloV5(path="yolo/train/labels",
        path_to_images="../images", 
        img_ext="jpg",
        cat_names=categories
    )
dataset.df.head(3)

Importing YOLO files...: 100%|██████████| 6005/6005 [28:07<00:00,  3.56it/s]


,img_folder,img_filename,img_path,img_id,img_width,img_height,img_depth,ann_segmented,ann_bbox_xmin,ann_bbox_ymin,...,ann_segmentation,ann_iscrowd,ann_pose,ann_truncated,ann_difficult,cat_id,cat_name,cat_supercategory,split,annotated
id,,,,,,,,,,,,,,,,,,,,,
0,../images,000411001ff7dd4f.jpg,,2,640,640,3,,110.72512,126.48128,...,,,,,,10,Chicken,,,1
1,../images,000411001ff7dd4f.jpg,,2,640,640,3,,287.82272,166.91264,...,,,,,,10,Chicken,,,1
2,../images,000411001ff7dd4f.jpg,,2,640,640,3,,596.65664,255.76320,...,,,,,,10,Chicken,,,1


In [11]:
print(dataset.analyze.classes)
#Create a tag for each class and store then in a dict where the class name is the key
tags = {}
for class_name in dataset.analyze.classes:
    tag = trainer.create_tag(project.id, class_name)
    tags[class_name] = tag

['Bear', 'Brown bear', 'Bull', 'Butterfly', 'Camel', 'Canary', 'Caterpillar', 'Cattle', 'Centipede', 'Cheetah', 'Chicken', 'Crab', 'Crocodile', 'Deer', 'Duck', 'Eagle', 'Elephant', 'Fish', 'Fox', 'Frog', 'Giraffe', 'Goat', 'Goldfish', 'Goose', 'Hamster', 'Harbor seal', 'Hedgehog', 'Hippopotamus', 'Horse', 'Jaguar', 'Jellyfish', 'Kangaroo', 'Koala', 'Ladybug', 'Leopard', 'Lion', 'Lizard', 'Lynx', 'Magpie', 'Monkey', 'Moths and butterflies', 'Mouse', 'Mule', 'Ostrich', 'Otter', 'Owl', 'Panda', 'Parrot', 'Penguin', 'Pig', 'Polar bear', 'Rabbit', 'Raccoon', 'Raven', 'Red panda', 'Rhinoceros', 'Scorpion', 'Sea lion', 'Sea turtle', 'Seahorse', 'Shark', 'Sheep', 'Shrimp', 'Snail', 'Snake', 'Sparrow', 'Spider', 'Squid', 'Squirrel', 'Starfish', 'Swan', 'Tick', 'Tiger', 'Tortoise', 'Turkey', 'Turtle', 'Whale', 'Woodpecker', 'Worm', 'Zebra']


In [14]:
#Iterate the rows for each image in the dataframe
for img_filename, img_df in dataset.df.groupby('img_filename'):
    img_path = str(PurePath(dataset.path_to_annotations, str(img_df.iloc[0].img_folder), img_filename))
    assert exists(img_path), f"File does not exist: {img_path}"

    #Create a region object for each bounding box in the dataset 
    regions = []
    for index, row in img_df.iterrows():

        #Normalize the boundings box coordinates between 0 and 1
        x = Decimal(row.ann_bbox_xmin / row.img_width).min(1)
        y = Decimal(row.ann_bbox_ymin / row.img_height).min(1)
        w = Decimal(row.ann_bbox_width / row.img_width).min(1-x)
        h = Decimal(row.ann_bbox_height / row.img_height).min(1-y)

        regions.append(Region(
                tag_id=tags[row.cat_name].id, 
                left=x,
                top=y,
                width=w,
                height=h
            )
        )
        print(regions)

    #Create an object with the image and all of the annotations for that image
    with open(img_path, mode="rb") as image_contents:
        image_and_annotations = [ImageFileCreateEntry(name=img_filename, contents=image_contents.read(), regions=regions)]

    #Upload the image and all annnotations for that image
    upload_result = trainer.create_images_from_files(
            project.id, 
            ImageFileCreateBatch(images=image_and_annotations)
        )
    
    #If upload is not successful, print details about that image for debugging 
    print(upload_result)
    if not upload_result.is_batch_successful:
        print("Image upload failed.")
        
        for image in upload_result.images:
            print(img_path)
            print("Image status: ", image.status)
            print(regions)
            

#This will take a few minutes 
print("Upload complete")

[<azure.cognitiveservices.vision.customvision.training.models._models_py3.Region object at 0x7f7b93e28850>, <azure.cognitiveservices.vision.customvision.training.models._models_py3.Region object at 0x7f7b93e36110>]
[<azure.cognitiveservices.vision.customvision.training.models._models_py3.Region object at 0x7f7b93e28850>, <azure.cognitiveservices.vision.customvision.training.models._models_py3.Region object at 0x7f7b93e36110>, <azure.cognitiveservices.vision.customvision.training.models._models_py3.Region object at 0x7f7b93e37100>]
{'additional_properties': {}, 'is_batch_successful': True, 'images': [<azure.cognitiveservices.vision.customvision.training.models._models_py3.ImageCreateResult object at 0x7f7b93e37af0>]}
{'additional_properties': {}, 'is_batch_successful': True, 'images': [<azure.cognitiveservices.vision.customvision.training.models._models_py3.ImageCreateResult object at 0x7f7b93e369e0>]}
{'additional_properties': {}, 'is_batch_successful': True, 'images': [<azure.cognitiv

In [26]:
import os
import pandas as pd
from tqdm import tqdm
from pylabel import importer

# Set the path to the extracted dataset
dataset_path = "data/extract_trainingimages/"

# Create an empty DataFrame to store the annotations

annotations = pd.DataFrame(columns=["image_path", "x_min", "y_min", "x_max", "y_max",  "class"])
#"img_width", "img_height", "ann_bbox_ymin", "ann_bbox_width", "nn_bbox_height",

# Iterate over the animal categories
for animal_category in tqdm(os.listdir(dataset_path)):
    category_dir = os.path.join(dataset_path, animal_category)
    label_dir = os.path.join(category_dir, "Label")

    # Iterate over the image files and their corresponding labels
    for image_file in os.listdir(category_dir):
        if image_file.endswith(".jpg"):
            image_path = os.path.join(category_dir, image_file)
            label_file = os.path.join(label_dir, os.path.splitext(image_file)[0] + ".txt")

            # Read the label file and extract the annotations
            
            with open(label_file, "r") as f:
                
                for line in f:
                    #class_id, x, y, w, h = line.split()
                    #print(line.split())
                    
                    if len(line.split()) == 5:
                        class_id = line.split()[0]
                        x = line.split()[1]
                        y = line.split()[2]
                        w = line.split()[3]
                        h = line.split()[4]
                    elif len(line.split()) == 6:
                        class_id = line.split()[0] + "_" + line.split()[1]
                        x = line.split()[-4]
                        y = line.split()[-3]
                        w = line.split()[-2]
                        h = line.split()[-1]
                    else:
                        class_id = line.split()[0] + "_" + line.split()[1] + "_" + line.split()[2]
                        x = line.split()[-4]
                        y = line.split()[-3]
                        w = line.split()[-2]
                        h = line.split()[-1]

                    x_min = float(x) - float(w) / 2
                    y_min = float(y) - float(h) / 2
                    x_max = float(x) + float(w) / 2
                    y_max = float(y) + float(h) / 2
                    
                    img = {
                            "image_path": image_path,
                            "x_min": x_min,
                            "y_min": y_min,
                            "x_max": x_max,
                            "y_max": y_max, 
                            #"img_width":img_width, 
                            #"img_height": img_height, 
                            #"ann_bbox_ymin":ann_bbox_ymin, 
                            #"ann_bbox_width": ann_bbox_width, 
                            #"nn_bbox_height":nn_bbox_height,
                            "class": animal_category,
                        }
                    extended = pd.DataFrame([img])
                    annotations = pd.concat([annotations,extended])



 51%|█████▏    | 41/80 [00:26<00:24,  1.56it/s]


KeyboardInterrupt: 

In [ ]:
annotations.reset_index(drop=True, inplace=True)
annotations.head()

In [ ]:
from azureml.core import Dataset
# Convert the pandas DataFrame to a PyLabel dataset
dataset = Dataset(annotations)

# Access and manipulate the dataset
dataset.head(5)

#### Import to Custom Vision

In [ ]:
annotations.groupby('image_path').head()

In [ ]:
tags = {}

classes = dataset['class'].unique()
for class_name in classes:
    tag = trainer.create_tag(project.id, class_name)
    tags[class_name] = tag

    
annotations.shape

In [ ]:
#Iterate the rows for each image in the dataframe
for image_path, img_df in annotations.groupby('image_path'):
    img_path = image_path
    assert exists(img_path), f"File does not exist: {img_path}"

    #Create a region object for each bounding box in the dataset 
    regions = []
    for index, row in img_df.iterrows():

        print(row)

        #Normalize the boundings box coordinates between 0 and 1
        x = Decimal(row.x_min / row.img_width).min(1)
        y = Decimal(row.ann_bbox_ymin / row.img_height).min(1)
        w = Decimal(row.ann_bbox_width / row.img_width).min(1-x)
        h = Decimal(row.ann_bbox_height / row.img_height).min(1-y)
        
        regions.append(Region(
                tag_id=tags[row.cat_name].id, 
                left=x,
                top=y,
                width=w,
                height=h
            )
        )

    #Create an object with the image and all of the annotations for that image
    with open(img_path, mode="rb") as image_contents:
        image_and_annotations = [ImageFileCreateEntry(name=image_path, contents=image_contents.read(), regions=regions)]

    #Upload the image and all annnotations for that image
    upload_result = trainer.create_images_from_files(
            project.id, 
            ImageFileCreateBatch(images=image_and_annotations)
        )
    #If upload is not successful, print details about that image for debugging 
    if not upload_result.is_batch_successful:
        print("Image upload failed.")
        for image in upload_result.images:
            print(img_path)
            print("Image status: ", image.status)
            print(regions)

#This will take a few minutes 
print("Upload complete")